# Load Data into our DB

In [6]:
import psycopg2
from random_word import RandomWords
import random
import pprint
pp = pprint.PrettyPrinter(indent=4)

r = RandomWords()


In [2]:
languages = ['python', 'javascript', 'java']

In [3]:
conn = psycopg2.connect(database="search",
                        host="localhost",
                        user="ross",
                        password="abc123",
                        port="5432")

In [8]:
# Add a little "real" data that we can search for

cur = conn.cursor()

for language in languages:
    api_name = f'stripe-{language}-api'
    api_version = 1
    internal_version = 1
    api_base_url = f'www.api.stripe.com'
    company = f'Stripe inc.'
    path_to_zip = '/data/zips'
    programming_language = language
    openapi_url = 'www.google.com'
    openapi_hash = 'abc123'

    # insert data
    cur.execute(f"INSERT INTO metadata(api_name, api_version, internal_version, api_base_url, company, path_to_zip, programming_language, openapi_url, openapi_hash) VALUES('{api_name}', '{api_version}', '{internal_version}', '{api_base_url}', '{company}', '{path_to_zip}', '{programming_language}', '{openapi_url}', '{openapi_hash}')");

conn.commit()
cur.close()

In [7]:

# insert data in bulks of 10, do this 100000 times for 1 million rows
for i in range(100000):
    print('first loop', i)
    cur = conn.cursor()
    for j in range(10):
        randomBase = r.get_random_word()
        randomBase2 = r.get_random_word()
        api_name = f'{randomBase}-{randomBase2}-api'
        api_version = 1
        internal_version = 1
        api_base_url = f'www.{randomBase}-{randomBase2}-api.com'
        company = f'{randomBase} {randomBase2} inc.'
        path_to_zip = '/data/zips'
        programming_language = random.choice(languages)
        openapi_url = 'www.google.com'
        openapi_hash = 'abc123'

        # insert data
        cur.execute(f"INSERT INTO metadata(api_name, api_version, internal_version, api_base_url, company, path_to_zip, programming_language, openapi_url, openapi_hash) VALUES('{api_name}', '{api_version}', '{internal_version}', '{api_base_url}', '{company}', '{path_to_zip}', '{programming_language}', '{openapi_url}', '{openapi_hash}')");

    print('inserting data')
    conn.commit()
    cur.close()

first loop 0
inserting data
first loop 1
inserting data
first loop 2
inserting data
first loop 3
inserting data
first loop 4
inserting data
first loop 5
inserting data
first loop 6
inserting data
first loop 7
inserting data
first loop 8
inserting data
first loop 9
inserting data
first loop 10
inserting data
first loop 11
inserting data
first loop 12
inserting data
first loop 13
inserting data
first loop 14
inserting data
first loop 15
inserting data
first loop 16
inserting data
first loop 17
inserting data
first loop 18
inserting data
first loop 19
inserting data
first loop 20
inserting data
first loop 21
inserting data
first loop 22
inserting data
first loop 23
inserting data
first loop 24
inserting data
first loop 25
inserting data
first loop 26
inserting data
first loop 27
inserting data
first loop 28
inserting data
first loop 29
inserting data
first loop 30
inserting data
first loop 31
inserting data
first loop 32
inserting data
first loop 33
inserting data
first loop 34
inserting 

KeyboardInterrupt: 

In [4]:
cur.close()

# Searching

This would be what our frontend hits when it sends a request to /search

In [12]:
searchString = 'stripe python'

In [13]:

cur = conn.cursor()

sql = """
SELECT api_name, company, programming_language, rank_ts_search, similarity
FROM
    metadata,
    phraseto_tsquery('english', '{searchString}') query,
    NULLIF(ts_rank(ts, query), 0) rank_ts_search,
    SIMILARITY('{searchString}', search_doc) similarity
WHERE ts @@ query OR similarity > 0
ORDER BY rank_ts_search DESC, similarity DESC
LIMIT 6;
"""

cur.execute(sql)
rows = cur.fetchall()
conn.commit()
cur.close()

pp.pprint(rows)

[   ('humour-hamstring-api', 'humour hamstring inc.', 'java', None, 0.13513513),
    ('backstring-marse-api', 'backstring marse inc.', 'python', None, 0.125),
    (   'seabag-intercorrelating-api',
        'seabag intercorrelating inc.',
        'python',
        None,
        0.11111111),
    (   'serenities-endangering-api',
        'serenities endangering inc.',
        'python',
        None,
        0.10869565),
    (   'stravaigs-arousing-api',
        'stravaigs arousing inc.',
        'java',
        None,
        0.102564104),
    (   'straightforward-cumbering-api',
        'straightforward cumbering inc.',
        'python',
        None,
        0.10204082)]


In [ ]:
conn.close()